In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, box, Polygon
import numpy as np
import matplotlib.pyplot as plt
import ast
import datetime
from sklearn.model_selection import TimeSeriesSplit
import calendar

In [2]:
def setupApiFile():
    apiFile = pd.read_csv("../data/processed/crash_street_api_features.csv")

    # remove columns we basically already have``
    apiFile.drop(columns={"maxspeed", "road_type", "AADT"}, inplace=True)

    # just print out before filterin
    print(f"With and without lane count, we have: {apiFile.shape[0]} crashes")

    apiFile = apiFile.dropna(subset=["lane_count"])

    apiFile.fillna(0, inplace=True)

    print(f"With only lane count, we have: {apiFile.shape[0]} crashes")

    for index, crash in apiFile.iterrows():
        try:
            apiFile.at[index, "lane_count"] = int(crash["lane_count"])
        except:
            laneCount = ast.literal_eval(crash["lane_count"])
            laneCount = int(laneCount[0]) + int(laneCount[1])
            apiFile.at[index, "lane_count"] = laneCount
    return apiFile

In [ ]:
apiFile = setupApiFile()
crash_frame = pd.read_csv("../data/processed/encoded_data_binary_encoding.csv")

# API already has this
crash_frame.drop(columns="Adjusted Average Daily Traffic Amount",inplace=True)

gdf = gpd.GeoDataFrame(
    crash_frame, 
    geometry=gpd.points_from_xy(crash_frame["Longitude"], crash_frame["Latitude"]), 
    crs="EPSG:4326"
)

# left upper bound - 33°15'08.7"N 97°21'46.3"W
# right lower bound - 32°22'36.5"N 96°07'37.0"W

#using our bounds, create the corners we'll use in our polygon
upperLeft = (-1 * (97 + 21/60 + 46.3/3600), 33 + 15/60 + 8.7/3600)
bottomRight = (-1 * (96 + 7/60 + 37/3600), 32 + 22/60 + 36.5/3600)

upperRight = (bottomRight[0], upperLeft[1])
bottomLeft = (upperLeft[0], bottomRight[1])

# create the polygon
dallasBounds = Polygon([upperLeft, upperRight, bottomRight, bottomLeft, upperLeft])

# create teh dataframe
dallasFrame = gpd.GeoDataFrame({"geometry": [dallasBounds]}, crs="EPSG:4326")

# cahnge coordinate system of both
dallasFrame = dallasFrame.to_crs(epsg=32614)
gdf = gdf.to_crs(epsg=32614)

cell_size = 500

minx, miny, maxx, maxy = dallasFrame.total_bounds

grids = []
index = 0
# create grids based on the bounds
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        grids.append(box(x, y, x+cell_size, y+cell_size))

# 2. Optional: intersect with Dallas boundary to crop
grid = gpd.GeoDataFrame({"geometry": grids}, crs=dallasFrame.crs)

grid = gpd.overlay(grid, dallasFrame, how="intersection")

# combine it based on which cells are matching. Now this is an array of cells
joined_data = gpd.sjoin(gdf, grid, how="left", predicate="intersects")

# drops crashes that don't correspond to a cell
joined_data = joined_data.drop(joined_data[joined_data["index_right"].isna()].index)

missingCells = []

for index, theGrid in grid.iterrows():
    if index not in joined_data["index_right"].values:
        new_row = gpd.GeoDataFrame([theGrid], crs=grid.crs)
        new_row["index_right"] = index
        missingCells.append(new_row)

if missingCells:
    missingCells = pd.concat(missingCells, ignore_index = True)
    joined_data = pd.concat([joined_data, missingCells], ignore_index=True)

print("Finished concatenating")


noCrashIndices = 0


# then, join by api data
joined_data = pd.merge(joined_data, apiFile, on=["Latitude", "Longitude"], how="left")


joined_data.drop(columns=["Crash ID"], inplace=True)

final_cells = []

hoursPerPeriod = 4

for i in range(0, 24, hoursPerPeriod):
    joined_data[f"time_bin_{i}_{i + 3}"] = 0

joined_data["crash_count_7d"] = 0
joined_data["crash_count_30d"] = 0

# they both are functionally the same, so rename for easier mapping
joined_data.rename(columns = {"index_right": "cell_id"}, inplace=True)
joined_data["label"] = 0

joined_data["Crash Date"] = pd.to_datetime(joined_data["Crash Date"], format="%Y-%m-%d")

joined_data = joined_data.to_crs("EPSG:4326")

final_cells = []

With and without lane count, we have: 41632 crashes
With only lane count, we have: 33649 crashes
Finished concatenating


In [ ]:
def getRollingCounts(theCrash, allCrashes, startIndex):
    rolling_7_count = 0
    rolling_30_count = 0
    # loop through previous crashes
    for z in range(startIndex - 1, -1, -1):
        prevCrashDate = allCrashes.iloc[z]["Crash Date"]
        daysPassed = (theCrash["Crash Date"] - prevCrashDate).days
        # use this if sattement to add it
        if(daysPassed == 0):
            prevHour = allCrashes.iloc[z]["Hour of Day"]
            if(prevHour < theCrash["Hour of Day"]):
                rolling_7_count += 1
                rolling_30_count += 1
        if(daysPassed > 30):
            break
        # note that we don't use an else statement. This way if both are true, it'll be added to both
        if(daysPassed <= 7):
            rolling_7_count += 1
        if(daysPassed <= 30):
            rolling_30_count += 1
    
    return (rolling_7_count, rolling_30_count)

def processCell(currentCrashes, coordinates, columns):
    returnedRows = []
    for year in [2023, 2024, 2025]:
        for month in range (1, 13, 1):
            for day in range(1, calendar.monthrange(year, month)[1] + 1, 1):
                for hour in range(0, 24, 1):
                    currentDate = datetime(year, month, day)
                    matchingSet = currentCrashes[(currentCrashes["Crash Date"] == currentDate) & (currentCrashes["Hour Of Day"] == hour)]
                    # if there is not a crash
                    if len(matchingSet) != 0:
                        for i, value in matchingSet.iterrows():
                            value["label"] = 1
                            rollingCounts = getRollingCounts(value, currentCrashes, i)
                            newRow["crash_count_7d"] = rollingCounts[0]
                            newRow["crash_count_30d"] = rollingCounts[1]
                    else:
                        newRow = pd.Series(index = columns, dtype=object)
                        newRow["Crash Date"] = pd.to_datetime(currentDate, format="%Y-%m-%d")
                        newRow["Hour Of Day"] = hour
                        newRow["Longitude"] = coordinates[0]
                        newRow["Latitude"] = coordinates[1]
                        newRow["label"] = 0
                        rollingCounts = getRollingCounts(value, currentCrashes, currentCrashes.shape[0])
                        newRow["crash_count_7d"] = rollingCounts[0]
                        newRow["crash_count_30d"] = rollingCounts[1]
    return returnedRows
                        


# group by when they go to similar cells
for _, crashes in joined_data.groupby("cell_id"):
    cellGeometry = list(crashes.iloc[0]["geometry"].exterior.coords)
    longitude = (cellGeometry[0][0] - cellGeometry[2][0]) / 2
    latitude = (cellGeometry[0][1] + cellGeometry[2][1]) / 2
    centerCoords = (longitude, latitude)

    cellIndex = crashes.iloc[0]["cell_id"]

    if(not pd.isna(crashes.iloc[0]["Latitude"])):
        print(f'In cell {cellIndex} we have {len(crashes)} crashes!')

    crashes.drop(columns=["geometry"], inplace=True)
    crashes = crashes.sort_values(["Crash Date", "Hour of Day"], ascending=True).reset_index(drop=True)

    columns = crashes[0].values.index

    # if empty row, make it an empty dataframe
    if(pd.isna(crashes[0]["latitude"])):
        crashes.drop(0, inplace=True)
    
    addedCrashes = processCell(crashes, centerCoords, columns)
    final_cells.extend(addedCrashes)


[(-97.36287051743017, 32.96306057130739), (-97.36290338822646, 32.961278598865434), (-97.36290325810803, 32.96306100051666), (-97.36287051743017, 32.96306057130739)]
[(-97.36278734176727, 32.967568938294924), (-97.36287051743017, 32.96306057130739), (-97.36290325810803, 32.96306100051666), (-97.36290292184492, 32.96757045362001), (-97.36278734176727, 32.967568938294924)]
[(-97.362704147562, 32.97207730183697), (-97.36278734176727, 32.967568938294924), (-97.36290292184492, 32.96757045362001), (-97.3629025754538, 32.972079903595244), (-97.362704147562, 32.97207730183697)]
[(-97.36262093481047, 32.97658566193324), (-97.362704147562, 32.97207730183697), (-97.3629025754538, 32.972079903595244), (-97.36290221893363, 32.976589350442104), (-97.36262093481047, 32.97658566193324)]
[(-97.36253770350888, 32.98109401858346), (-97.36262093481047, 32.97658566193324), (-97.36290221893363, 32.976589350442104), (-97.3629018522834, 32.98109879416036), (-97.36253770350888, 32.98109401858346)]
[(-97.362454

KeyboardInterrupt: 

In [ ]:
exportedDf = pd.DataFrame(final_cells)

# Removed b/c of time series
# exportedDf.drop(columns=["Hour of Day", "Crash Date"], inplace=True)

exportedDf.to_csv("../data/final/true_preprocessed_data.csv", index=False)

In [ ]:
numCrashGroups = 0
numCrashesTotal = 0

# group by when they go to similar cells
for _, crashes in joined_data.groupby("index_left"):
    
    numCrashGroups+= 1
    numCrashesTotal += len(crashes)
mean = numCrashesTotal / numCrashGroups

standardDev = 0

# group by when they go to similar cells
for _, crashes in joined_data.groupby("index_left"):
    standardDev += (len(crashes) - mean)**2
standardDev /= (numCrashGroups - 1)

standardDev = standardDev ** (1/2)

print(f"Mean crashes was {mean}, standard deviation was {standardDev}")

Mean crashes was 1.9060398531292546, standard deviation was 199.48931918548627
